# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import numpy as n
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sinabang,2.4803,96.3801,80.98,79,98,2.28,ID,1690417900
1,1,gondal,21.9667,70.8000,77.41,94,100,9.84,IN,1690417925
2,2,port-aux-francais,-49.3500,70.2167,40.03,77,90,38.70,TF,1690417925
3,3,mermoz boabab,14.7065,-17.4758,83.98,79,0,0.00,SN,1690417925
4,4,avarua,-21.2078,-159.7750,75.25,64,100,8.05,CK,1690417925


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
all_city_map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<80,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>60,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<10,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,uruacu,-14.5247,-49.1408,74.43,31,0,3.02,BR,1690417713
14,14,kone,-21.0595,164.8658,75.38,39,0,8.88,NC,1690417927
38,38,palmas,-10.2128,-48.3603,75.07,60,0,3.44,BR,1690417860
46,46,colonia,50.9333,6.9500,63.97,70,0,9.22,DE,1690417696
68,68,beyneu,45.3167,55.2000,78.64,39,0,9.80,KZ,1690417941
102,102,yeraliyev,43.2000,51.6500,75.67,53,0,7.29,KZ,1690417948
161,161,itacoatiara,-3.1431,-58.4442,79.18,56,0,0.63,BR,1690417959
193,193,fes,34.0372,-4.9998,71.85,60,0,3.44,MA,1690417965
217,217,bermeo,43.4209,-2.7215,67.24,89,0,4.23,ES,1690417969
267,267,melouza,35.9800,4.1867,73.09,50,0,6.08,DZ,1690417986


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,uruacu,BR,-14.5247,-49.1408,31,
14,kone,NC,-21.0595,164.8658,39,
38,palmas,BR,-10.2128,-48.3603,60,
46,colonia,DE,50.9333,6.9500,70,
68,beyneu,KZ,45.3167,55.2000,39,
102,yeraliyev,KZ,43.2000,51.6500,53,
161,itacoatiara,BR,-3.1431,-58.4442,56,
193,fes,MA,34.0372,-4.9998,60,
217,bermeo,ES,43.4209,-2.7215,89,
267,melouza,DZ,35.9800,4.1867,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
uruacu - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
palmas - nearest hotel: Hotel Triângulo Mineiro
colonia - nearest hotel: Wasserturm Hotel Cologne
beyneu - nearest hotel: Johan yu
yeraliyev - nearest hotel: Kuryk
itacoatiara - nearest hotel: Regine's Hotel
fes - nearest hotel: Maghreb Hôtel
bermeo - nearest hotel: Aldatzeta ostatua
melouza - nearest hotel: No hotel found
uspenskoye - nearest hotel: No hotel found
darwin - nearest hotel: Mantra Pantanas Darwin
amarante do maranhao - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa
le blanc - nearest hotel: Chez P'tit Jean
bordj zemoura - nearest hotel: No hotel found
yashkul' - nearest hotel: No hotel found
borzya - nearest hotel: гостиница Глория
paphos - nearest hotel: Mayfair Hotel
yarmouth - nearest hotel: The Inn At Cape Cod
ulladulla - nearest hotel: Hotel Marlin
irymple - nearest hotel: Chaffey Motor Inn
nelson bay - nearest hotel: No hotel found
la rinconada - n

,City,Country,Lat,Lng,Humidity,Hotel Name
7,uruacu,BR,-14.5247,-49.1408,31,No hotel found
14,kone,NC,-21.0595,164.8658,39,Pacifik Appartels
38,palmas,BR,-10.2128,-48.3603,60,Hotel Triângulo Mineiro
46,colonia,DE,50.9333,6.9500,70,Wasserturm Hotel Cologne
68,beyneu,KZ,45.3167,55.2000,39,Johan yu
102,yeraliyev,KZ,43.2000,51.6500,53,Kuryk
161,itacoatiara,BR,-3.1431,-58.4442,56,Regine's Hotel
193,fes,MA,34.0372,-4.9998,60,Maghreb Hôtel
217,bermeo,ES,43.4209,-2.7215,89,Aldatzeta ostatua
267,melouza,DZ,35.9800,4.1867,50,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)